# Machine Learning

## Modulos a emplear

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, scale, StandardScaler
from sklearn.decomposition import PCA
import sys
import os

# Funciones importadas como modulos
notebook_dir = os.getcwd()
scripts_dir = os.path.join(notebook_dir, "..", "src")
sys.path.append(scripts_dir)
from data_transform.print_unique_values import print_unique_values

# Codigo para imprimir mas de una salida de la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Carga y selección de la Información

### Seleccionamos la informacion del pozo clave y la editamos

In [2]:
# Definimos la ruta de la informacion procesada
file_processed = "data_pijije_rhob_nphi_dtco_syn.csv"
path_data_processed = os.path.join(notebook_dir, "..", "data", "processed", file_processed)

# Cargamos la información procesada
pijije_df = pd.read_csv(path_data_processed)

# Mostramos los pozos en el DataFrame
print_unique_values(pijije_df, "wellname")

Valores únicos en la columna 'wellname': Caparroso-71, Caparroso-75, Pijije-101, Pijije-111, Pijije-112, Pijije-117, Pijije-125, Pijije-127, Pijije-1A, Pijije-21, Pijije-25, Pijije-25REE2, Pijije-27, Pijije-31REE, Pijije-33, Pijije-34, Pijije-41, Pijije-42


In [3]:
# Seleccionamos el pozo/los pozos de interes
key_wells_lst = ["Caparroso-75"]
# Creamos el DataFrame key_well con la informacion de interes
key_well = pijije_df[pijije_df["wellname"].isin(key_wells_lst)].copy()

# Definimos las columnas de interes
key_well_tops = ["cretacico", "jurasico"]
features = ["gr", "rp", "rhob", "nphi"]
target = ["dtco"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

## Modelo de Machine Learning

In [4]:
# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')


RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.965/2.410


## Aplicamos el modelo de Machine Learning

In [5]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
well_target = "Pijije-41"
predict_well = pijije_df[pijije_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

## Guardamos el modelo de Machine Learning

In [6]:
# Guardamos el registro sintetico en formato csv
synthetic_file_name = f"{well_target}_{target[0]}_syn.csv"
path_to_save = os.path.join(notebook_dir, "..", "data", "processed", synthetic_file_name)
predict_well[["wellname", "md", target[0] +"_syn"]].to_csv(path_to_save, index=False)